In [ ]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import cv2
import keras
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D,BatchNormalization, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img


In [ ]:
# Create Dataset

def create_data(data_dir,class_folder):
  directory = data_dir + "/" + class_folder + "/"
  data = os.listdir(directory)
  random.shuffle(data)
  train = data[:100]
  test = data[100:]

  return train,test

class_folder = []
path = "/content/drive/MyDrive/Dataset/15-Scene/15-Scene"

for category in sorted(os.listdir(path)):
  class_folder.append(category)

train_set = []
test_set = []

data_dir = '/content/drive/MyDrive/Dataset/15-Scene/15-Scene'

for class_name in class_folder:
  train,test = create_data(data_dir,class_name)
  train_data = [class_name,train]
  test_data = [class_name,test]
  train_set.append(train_data)
  test_set.append(test_data)

train_set = np.array(train_set,dtype=object)
test_set = np.array(test_set,dtype=object)

In [ ]:
# Create directory to store train and test folder

create_dir = "/content/drive/MyDrive/Dataset/Train_Test"
output_directory = os.mkdir(os.path.join(create_dir,'Scene15_Output'))

FileExistsError: ignored

In [ ]:
def write_img_to_folder(data_dir,output_path,output_directory,folder_name,dataset):

  new_folder = os.mkdir(os.path.join(output_directory,folder_name))

  for data in dataset:
    class_name = data[0]
    data_files = data[1]

    data_path = os.path.join(output_path + "/" + str(folder_name) ,class_name)
    category_dire = os.mkdir(data_path)

    for filename in data_files:
      img = cv2.imread(data_dir + "/" + class_name + "/" + filename, 0)
      cv2.imwrite(os.path.join(data_path, filename), img)

output_path = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output"
output_directory = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output"

# Train/Test folder using write_img_to_folder function

train_folder = write_img_to_folder(data_dir,output_path,output_directory,'Train', train_set)
test_folder = write_img_to_folder(data_dir, output_path,output_directory,'Test', test_set)

FileExistsError: ignored

In [ ]:
def convert_img_to_array(dir_path):
    X = []
    y = []
    one_hot_lookups = np.eye(15)

    for category in sorted(os.listdir(dir_path)):

      for files in os.listdir(dir_path + category):

        img = load_img(dir_path + category + '/' + files,target_size=(224, 224))
        #img = cv2.resize(img, dsize = (227,227))
        img = img_to_array(img) 
        img = tf.image.per_image_standardization(img)  
        X.append(img) 
        y.append(np.reshape(one_hot_lookups[int(category)],[15]))  

    return X,y

In [ ]:
def load_data():
    train_dir_path = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output/Train/"
    X_train,y_train = convert_img_to_array(train_dir_path)
    
    test_dir_path = "/content/drive/MyDrive/Dataset/Train_Test/Scene15_Output/Test/"
    X_test, y_test = convert_img_to_array(test_dir_path)
    
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    X_train /= np.std(X_train, axis = 0)
    X_test /= np.std(X_test, axis = 0)
    
    return ((X_train, y_train), (X_test, y_test))
(x_train, y_train), (x_test, y_test) = load_data()


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1500, 224, 224, 3)
(1500, 15)
(2985, 224, 224, 3)
(2985, 15)


In [ ]:
model = Sequential()
# 1st layer..........
model.add(Conv2D(96, kernel_size = (11, 11), strides = (4,4), activation='relu', padding = 'valid' , data_format='channels_last', input_shape=(224, 224, 3),name = "first"))
model.add(MaxPooling2D(pool_size=(3,3) , strides = (2,2) , padding = 'valid' )) 
model.add(BatchNormalization())

# 2nd layer................
model.add(Conv2D(256, kernel_size=(5,5), activation='relu', strides=(1,1), padding = 'valid', name = "second"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

#3rd layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "third"))
model.add(BatchNormalization())

#4th layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "fourth"))
model.add(BatchNormalization())

#5th layer.................
model.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), activation = 'relu', padding='valid' , name = "fifth"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))

# Fully Connected Layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(15, activation = 'softmax')) #As number of nodes in last layer in softmax is number of classes, where each node is probability of classes


# Compile the model
optimizer = SGD(learning_rate = 1e-3,momentum = 0.9)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.get_layer("second").get_weights()[0].shape)

(5, 5, 96, 256)


In [ ]:
import numpy as np
weights_dic = np.load('/content/drive/MyDrive/Dataset/bvlc_alexnet.npy', encoding='bytes',allow_pickle=True).item()

In [ ]:
model.get_layer("first").set_weights([weights_dic["conv1"][0], weights_dic["conv1"][1]])
# model.get_layer("second").set_weights([weights_dic["conv2"][0], weights_dic["conv2"][1]])
# model.get_layer("third").set_weights([weights_dic["conv3"][0], weights_dic["conv3"][1]])
# model.get_layer("fourth").set_weights([weights_dic["conv4"][0], weights_dic["conv4"][1]])
# model.get_layer("fifth").set_weights([weights_dic["conv5"][0], weights_dic["conv5"][1]])

In [ ]:
from time import time
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

iter_1 = model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 25, verbose = 1,callbacks=[tensorboard])

Epoch 1/25
47/47 [==============================] - 167s 4s/step - loss: 2.5820 - accuracy: 0.2667 - val_loss: 7.1001 - val_accuracy: 0.1725
Epoch 2/25
47/47 [==============================] - 192s 4s/step - loss: 1.7787 - accuracy: 0.4420 - val_loss: 3.6648 - val_accuracy: 0.3551
Epoch 3/25
47/47 [==============================] - 193s 4s/step - loss: 1.4510 - accuracy: 0.5167 - val_loss: 2.6940 - val_accuracy: 0.4379
Epoch 4/25
47/47 [==============================] - 194s 4s/step - loss: 1.3043 - accuracy: 0.5767 - val_loss: 1.9531 - val_accuracy: 0.5116
Epoch 5/25
47/47 [==============================] - 192s 4s/step - loss: 1.2488 - accuracy: 0.5880 - val_loss: 3.0479 - val_accuracy: 0.3876
Epoch 6/25
47/47 [==============================] - 164s 4s/step - loss: 1.1142 - accuracy: 0.6327 - val_loss: 2.2275 - val_accuracy: 0.4817
Epoch 7/25
47/47 [==============================] - 165s 4s/step - loss: 1.0106 - accuracy: 0.6767 - val_loss: 1.6596 - val_accuracy: 0.5685
Epoch 8/25
47

In [ ]:
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

iter_2 = model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 10, verbose = 1,callbacks=[tensorboard], shuffle = True)

In [ ]:
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

iter_3 = model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 10, verbose = 1,callbacks=[tensorboard], shuffle = True)

In [ ]:
y_pred = model.predict(x_test, y_test)

In [ ]:
print("Testing Accuracy: ",accuracy_score(y_test, y_pred) * 100, "%")